In [1]:
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycountry_convert as pc
import pycountry
from IPython.display import Image
import seaborn as sns
from networkx.drawing.nx_pydot import write_dot
from pathlib import Path
import pylab as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from collections import Counter

## 1. CoR_CoP

In [2]:
# load data
df_CoR_CoP = pd.read_csv('PATH\data\CoR-CoP_fractional_edges.csv').drop(columns = {'Unnamed: 0'})

In [3]:
## load g77 and OECD data, assign overlapping to g77
g77 = pd.read_csv('PATH\data\g77.csv')
oecd = pd.read_csv('PATH\data\oecd.csv')

delete_from_oecd = []

for i in list(g77['Code']): 
    if i in list(oecd['Code']):
        delete_from_oecd.append(i)

for i in list(g77['Code']): 
    if i in list(oecd['Code']):
        delete_from_oecd.append(i)        

oecd_clean = [x for x in list(oecd['Code']) if x not in delete_from_oecd]

econ_group_CoR = []
econ_group_CoP = []
for i in list(df_CoR_CoP['CoR']):
    if i in [pc.country_alpha3_to_country_alpha2(x) for x in oecd_clean]: 
        econ_group_CoR.append('OECD')
    elif i in ['RU','BR','IN','CN','ZA']:
        econ_group_CoR.append('BRICS')
    elif i in [pc.country_alpha3_to_country_alpha2(x) for x in g77['Code'] if x not in ['RU','BR','IN','CN','ZA']]:
        econ_group_CoR.append('G77')
    else: 
        econ_group_CoR.append('other')
        
for i in list(df_CoR_CoP['CoP']):
    if i in [pc.country_alpha3_to_country_alpha2(x) for x in oecd_clean]: 
        econ_group_CoP.append('OECD')
    elif i in ['RU','BR','IN','CN','ZA']:
        econ_group_CoP.append('BRICS')
    elif i in [pc.country_alpha3_to_country_alpha2(x) for x in g77['Code'] if x not in ['RU','BR','IN','CN','ZA']]:
        econ_group_CoP.append('G77')
    else: 
        econ_group_CoP.append('other')
    
df_CoR_CoP['econ_group_CoR'] = econ_group_CoR
df_CoR_CoP['econ_group_CoP'] = econ_group_CoP

In [8]:
df_CoR_CoP_flow_balance= df_CoR_CoP.drop_duplicates(subset=['CoR','CoP'])

In [36]:
df_CoR_CoP_provided_all = df_CoR_CoP.groupby(['CoR'])['weights'].sum().reset_index(name = 'provided knowledge flow').sort_values(by = ['provided knowledge flow'],ascending=False).rename(columns ={'CoR':'Country'})
df_CoR_CoP_used_all = df_CoR_CoP.groupby(['CoP'])['weights'].sum().reset_index(name = 'used knowledge flow').sort_values(by = ['used knowledge flow'],ascending=False).rename(columns ={'CoP':'Country'})
df_CoR_CoP_flow_all = pd.merge(df_CoR_CoP_provided_all, df_CoR_CoP_used_all, on = 'Country')
df_CoR_CoP_flow_all['balance'] = df_CoR_CoP_flow_all['provided knowledge flow']-df_CoR_CoP_flow_all['used knowledge flow']
df_CoR_CoP_flow_all['Knowledge provision balance_all'] = df_CoR_CoP_flow_all['provided knowledge flow']/df_CoR_CoP_flow_all['used knowledge flow']

In [37]:
#filter self-loops
to_del = []
for row in list(df_CoR_CoP.index):
    if df_CoR_CoP.iloc[row]['CoR'] == df_CoR_CoP.iloc[row]['CoP']: 
        to_del.append(row)

In [38]:
df_CoR_CoP_flow_balance = df_CoR_CoP.drop(to_del)

In [45]:
df_CoR_CoP_flow_balance_provided = df_CoR_CoP_flow_balance.groupby(['CoR'])['weights'].sum().reset_index(name = 'provided knowledge flow').sort_values(by = ['provided knowledge flow'],ascending=False).rename(columns ={'CoR':'Country'})
df_CoR_CoP_flow_balance_used = df_CoR_CoP_flow_balance.groupby(['CoP'])['weights'].sum().reset_index(name = 'used knowledge flow').sort_values(by = ['used knowledge flow'],ascending=False).rename(columns ={'CoP':'Country'})
df_CoR_CoP_flow = pd.merge(df_CoR_CoP_flow_balance_provided, df_CoR_CoP_flow_balance_used, on = 'Country')
df_CoR_CoP_flow['balance_sum'] = df_CoR_CoP_flow['provided knowledge flow']-df_CoR_CoP_flow['used knowledge flow']
df_CoR_CoP_flow['External knowledge provision balance'] = df_CoR_CoP_flow['provided knowledge flow']/df_CoR_CoP_flow['used knowledge flow']

In [48]:
df_CoR_CoP_flow['Knowledge provision balance'] = list(df_CoR_CoP_flow_all['Knowledge provision balance_all'])

In [50]:
df_CoR_CoP_flow.to_csv('PATH\data\CoR-CoP_flow.csv')